In [25]:
from __future__ import print_function                  # Allows for python3 printing
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, SimpleRNN, LSTM, GRU, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np
import sys

filename = "sonnet.txt"
#read the file and make all chracaters lowercase
text = open(filename).read().lower()

# summarize the loaded data
print('text length:', len(text))    # this prints the length of the text 

chars = sorted(list(set(text)))    # sorted():Return a new list containing all items from the iterable in ascending order.
print('total chars:', len(chars))
print(chars)
##This should print 38.

# 4.1 encoding and decoding dictionaries
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# 4.2 cut corpus into equal length sequences
maxlen = 40
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])               
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)   #generate zeros with size[number of sentences,40,38]
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) #generate zeros with size[number of sentences,38]
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

model = Sequential()
model.add(GRU(192, input_shape=(x.shape[1], x.shape[2])))
model.add(Dense(len(chars), activation='rmsprop'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

for iteration in range(1, 11):
    model.fit(x, y,
              batch_size=128,
              epochs=10)
    
# generating text
    print('\nIteration', iteration)
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]


    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]      #predict using model, generating a matrix of softmax
        next_index = np.argmax(preds)                    #we want the index with highest probability 
        next_char = indices_char[next_index]             #convert number back to character using the dictionary
        sentence = sentence[1:] + next_char              #append the character predicted to the sentence (start from the second character)

        sys.stdout.write(next_char)                
        sys.stdout.flush()
    print('\n')




text length: 95690
total chars: 38
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


ValueError: Unknown activation function:rmsprop